In [ ]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Lambda, Reshape
from keras import layers
import keras
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
LATENT_DIM = 100

# Define the sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
        
# Define the encoder network
encoder_inputs = tf.keras.Input(shape=(784,))
x = layers.Dense(128, activation="relu")(encoder_inputs)
z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# Define the decoder network
decoder = Sequential([
    InputLayer(shape=(LATENT_DIM,)),
    Dense(128, activation='relu'),
    Dense(784, activation='sigmoid')
])
def vae_loss(x, y):
    z_mean, z_log_var, z = encoder(x)
    reconstruction = decoder(z)
    reconstruction_loss = tf.reduce_mean(
        tf.reduce_sum(
            tf.keras.losses.binary_crossentropy(y, reconstruction),axis=-1
        )
    )
    kl_loss = -0.5 * tf.reduce_mean(
        1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    )
    total_loss = reconstruction_loss + kl_loss
    return total_loss

class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return  z_mean, z_log_var,reconstruction
       
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy( tf.cast(y, tf.float32) , reconstruction),axis=-1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }



vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 784)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128)       │    100,480 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 100)       │     12,900 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 100)       │     12,900 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling_4          │ (None, 100)       │          0 │ z_mean[0][0],     │
│ (Sampling)          │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 126,280 (493.28 KB)

 Trainable params: 126,280 (493.28 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#import 
import numpy as np

# Load MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 784))
x_test = np.reshape(x_test, (len(x_test), 784))

# Add noise to the data
x_train_noisy = x_train + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)


# # Train the model
vae.fit(x_train_noisy, x_train, epochs=10, batch_size=128, validation_data=(x_test_noisy, x_test))

# # Generate images from noise data
reconstructed_images = vae.predict(x_test_noisy)

# # Display some images
# import matplotlib.pyplot as plt

# n = 10  # Display how many images
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # Display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test_noisy[i].reshape(28, 28), cmap='gray')
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # Display reconstruction
#     ax = plt.subplot(2, n, i + 1 + n)
#     plt.imshow(reconstructed_images[i].reshape(28, 28), cmap='gray')
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - kl_loss: 1.9520 - loss: 45.7004 - reconstruction_loss: 43.7484

ValueError: No loss to compute. Provide a `loss` argument in `compile()`.

In [ ]:
print(x_train.shape)
import matplotlib.pyplot as plt
# Display original
ax = plt.subplot(2, 2,  1)
plt.imshow(x_train[0].reshape(28, 28), cmap='gray')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [ ]:
print(x_train_noisy.shape)
import matplotlib.pyplot as plt
# Display original
ax = plt.subplot(2, 2,  1)
plt.imshow(x_train_noisy[0].reshape(28, 28), cmap='gray')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [23]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]


In [24]:
import pandas as pd

def generate_input_data(strike_prices, current_price, dtm, put_volumes, call_volumes):
    data = {
        'Strike Price': strike_prices,
        'Current Price': [current_price] * len(strike_prices),
        'Days to Maturity': [dtm] * len(strike_prices),
        'Total Put Volume': put_volumes,
        'Total Call Volume': call_volumes
    }
    return pd.DataFrame(data)

# Example strike prices
strike_prices = [100, 125, 150, 135]
current_price = 120  # Example current price
dtm = 30  # Example days to maturity

# Example put and call volumes
put_volumes = [250, 300, 200, 150]  # Example total put volumes for each strike price
call_volumes = [200, 150, 250, 300]  # Example total call volumes for each strike price

# Generate input data
input_data = generate_input_data(strike_prices, current_price, dtm, put_volumes, call_volumes)

print("Example input data for VAE model:")
print(input_data)

Example input data for VAE model:
   Strike Price  Current Price  Days to Maturity  Total Put Volume  \
0           100            120                30               250   
1           125            120                30               300   
2           150            120                30               200   
3           135            120                30               150   

   Total Call Volume  
0                200  
1                150  
2                250  
3                300  
